In [3]:
import pandas as pd
import numpy as np
import itertools

In [5]:
df1 = pd.read_csv("./data/columns중구/중구컬럼4.csv")

In [9]:
df2 = pd.read_csv("./data/columns중구/중구컬럼5-6.csv")

In [4]:
df1.head()

,대여소자치구,대여소_ID,대여소번호,주간,시간대,대여건수,반납건수,총이용건수,잔여자전거수
0,중구,ST-116,300,주말,0,1,2,3,5
1,중구,ST-116,300,주말,1,1,1,2,7
2,중구,ST-116,300,주말,2,4,2,6,6
3,중구,ST-116,300,주말,3,0,0,0,5
4,중구,ST-116,300,주말,4,0,0,0,5


In [26]:
df1 = df1.drop(["대여소자치구"], axis = 1)

In [27]:
df2.head()

,대여소_ID,주간,시간대,동일대여소대여반납,중구내대여반납,유출량,유입량
0,ST-116,주말,0,0,0,1,2
1,ST-116,주말,1,0,0,1,1
2,ST-116,주말,2,0,1,3,0
3,ST-116,주말,5,0,0,0,1
4,ST-116,주말,7,0,2,2,10


In [28]:
df1.shape

(4318, 8)

In [29]:
df2.shape

(4022, 7)

In [30]:
# df2에 아예 존재하지 않는 대여소들 제거
remove_id = ['ST-128', 'ST-129', 'ST-139', 'ST-146', 'ST-1770', 'ST-1784', 'ST-1787',
             'ST-1788', 'ST-188', 'ST-190', 'ST-192', 'ST-193', 'ST-2523', 'ST-2524', 'ST-945']

In [31]:
df1 = df1[~df1["대여소_ID"].isin(remove_id)]

In [32]:
df1.shape

(4318, 8)

In [33]:
df2.shape

(4022, 7)

In [34]:
df1.head()

,대여소_ID,대여소번호,주간,시간대,대여건수,반납건수,총이용건수,잔여자전거수
0,ST-116,300,주말,0,1,2,3,5
1,ST-116,300,주말,1,1,1,2,7
2,ST-116,300,주말,2,4,2,6,6
3,ST-116,300,주말,3,0,0,0,5
4,ST-116,300,주말,4,0,0,0,5


In [35]:
df2.head()

,대여소_ID,주간,시간대,동일대여소대여반납,중구내대여반납,유출량,유입량
0,ST-116,주말,0,0,0,1,2
1,ST-116,주말,1,0,0,1,1
2,ST-116,주말,2,0,1,3,0
3,ST-116,주말,5,0,0,0,1
4,ST-116,주말,7,0,2,2,10


In [36]:
df1["대여소_ID"].nunique()

90

In [37]:
df2["대여소_ID"].nunique()

90

In [38]:
merged_df = pd.merge(df1, df2, on=["대여소_ID","주간","시간대"], how="left").fillna(0)

In [39]:
merged_df.shape

(4318, 12)

In [40]:
merged_df["동일대여소대여반납"] = merged_df["동일대여소대여반납"].astype(int)
merged_df["중구내대여반납"] = merged_df["중구내대여반납"].astype(int)
merged_df["유입량"] = merged_df["유입량"].astype(int)
merged_df["유출량"] = merged_df["유출량"].astype(int)

In [41]:
merged_df

,대여소_ID,대여소번호,주간,시간대,대여건수,반납건수,총이용건수,잔여자전거수,동일대여소대여반납,중구내대여반납,유출량,유입량
0,ST-116,300,주말,0,1,2,3,5,0,0,1,2
1,ST-116,300,주말,1,1,1,2,7,0,0,1,1
2,ST-116,300,주말,2,4,2,6,6,0,1,3,0
3,ST-116,300,주말,3,0,0,0,5,0,0,0,0
4,ST-116,300,주말,4,0,0,0,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4313,ST-977,391,평일,19,40,43,83,15,1,6,33,14
4314,ST-977,391,평일,20,21,17,38,19,1,4,16,7
4315,ST-977,391,평일,21,11,8,19,11,1,3,7,3
4316,ST-977,391,평일,22,10,15,25,12,0,2,8,4


In [42]:
all_combinations = pd.DataFrame(
    list(itertools.product(
        merged_df["대여소_ID"].unique(),
        merged_df["주간"].unique(),
        range(24)
    )),
    columns=["대여소_ID","주간","시간대"]
)

# 2. 기존 merged_df랑 outer join
merged_complete = (
    all_combinations
    .merge(merged_df, on=["대여소_ID","주간","시간대"], how="left")
    .fillna(0)   # 빠진 값은 전부 0으로 채움
)

# 3. 필요하다면 숫자형으로 변환
numeric_cols = merged_df.columns.drop(["대여소_ID","주간","시간대"])
merged_complete[numeric_cols] = merged_complete[numeric_cols].astype(int)

In [43]:
merged_complete.head(24)

,대여소_ID,주간,시간대,대여소번호,대여건수,반납건수,총이용건수,잔여자전거수,동일대여소대여반납,중구내대여반납,유출량,유입량
0,ST-116,주말,0,300,1,2,3,5,0,0,1,2
1,ST-116,주말,1,300,1,1,2,7,0,0,1,1
2,ST-116,주말,2,300,4,2,6,6,0,1,3,0
3,ST-116,주말,3,300,0,0,0,5,0,0,0,0
4,ST-116,주말,4,300,0,0,0,5,0,0,0,0
5,ST-116,주말,5,300,0,1,1,5,0,0,0,1
6,ST-116,주말,6,300,0,0,0,6,0,0,0,0
7,ST-116,주말,7,300,4,4,8,6,0,2,2,10
8,ST-116,주말,8,300,6,14,20,6,0,3,3,5
9,ST-116,주말,9,300,8,12,20,10,0,4,4,6


In [48]:
master = pd.read_csv("./data/master.csv", encoding = "cp949")

In [49]:
master = master.drop(["주소1", "주소2"], axis = 1)

In [52]:
loc_merge = pd.merge(
    merged_complete, master,
    on = ["대여소_ID"],
    how = "left"
)

In [53]:
loc_merge.head()

,대여소_ID,주간,시간대,대여소번호,대여건수,반납건수,총이용건수,잔여자전거수,동일대여소대여반납,중구내대여반납,유출량,유입량,위도,경도
0,ST-116,주말,0,300,1,2,3,5,0,0,1,2,37.56805,126.969231
1,ST-116,주말,1,300,1,1,2,7,0,0,1,1,37.56805,126.969231
2,ST-116,주말,2,300,4,2,6,6,0,1,3,0,37.56805,126.969231
3,ST-116,주말,3,300,0,0,0,5,0,0,0,0,37.56805,126.969231
4,ST-116,주말,4,300,0,0,0,5,0,0,0,0,37.56805,126.969231


In [57]:
loc_merge.to_csv("./data/merged_df.csv", index = False)

In [58]:
df = pd.read_csv("./data/merged_df.csv")

In [59]:
df

,대여소_ID,주간,시간대,대여소번호,대여건수,반납건수,총이용건수,잔여자전거수,동일대여소대여반납,중구내대여반납,유출량,유입량,위도,경도
0,ST-116,주말,0,300,1,2,3,5,0,0,1,2,37.568050,126.969231
1,ST-116,주말,1,300,1,1,2,7,0,0,1,1,37.568050,126.969231
2,ST-116,주말,2,300,4,2,6,6,0,1,3,0,37.568050,126.969231
3,ST-116,주말,3,300,0,0,0,5,0,0,0,0,37.568050,126.969231
4,ST-116,주말,4,300,0,0,0,5,0,0,0,0,37.568050,126.969231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,ST-977,평일,19,391,40,43,83,15,1,6,33,14,37.565052,126.973404
4316,ST-977,평일,20,391,21,17,38,19,1,4,16,7,37.565052,126.973404
4317,ST-977,평일,21,391,11,8,19,11,1,3,7,3,37.565052,126.973404
4318,ST-977,평일,22,391,10,15,25,12,0,2,8,4,37.565052,126.973404
